#Metodologia Aplicada

Etapa A — Pré‑processamento Completo:

- Carregamento e flatten de estruturas JSON hierárquicas em DataFrames relacionais
- Junção das três entidades preservando relacionamento vaga → prospects → candidatos
- Limpeza de duplicatas, normalização de strings e harmonização de valores categóricos
- Tratamento de valores ausentes com estratégias domain‑specific ("Desconhecido" para categóricas)
- One‑hot encoding controlado apenas em variáveis de baixa cardinalidade para evitar explosão dimensional
- Normalização Z‑score em features numéricas para compatibilidade com algoritmos de ML

Etapa B — Engenharia de Features Específicas:

- Match Técnico: tech_overlap_count calcula interseção de competências entre vaga e candidato usando NLP básico
- Sinalização SAP: sap_pair identifica sincronia entre demanda SAP da vaga e conhecimento do candidato
- Compatibilidade de Idiomas: ingles_ok/espanhol_ok avaliam se candidato atende requisitos mínimos de idioma
- Análise de Senioridade: senioridade_gap/senioridade_ok medem alinhamento de nível profissional
- Dinâmica do Funil: days_update/situacao_ord capturam velocidade e posição no processo seletivo
- Features de Interação: ok_eng_sen/len_cv_bin combinam sinais para capturar efeitos não‑lineares

Resultado Esperado Quantificado
Dataset Final:

- Tamanho: 53.759 registros (combinações únicas vaga‑candidato) × 21 colunas otimizadas
- Estrutura: 2 chaves relacionais + 19 features preditivas especializadas em recrutamento
- Qualidade: Dados limpos, tipos otimizados (int8, float32), sem valores ausentes tratados
- Formato: CSV UTF‑8

Features de Alta Qualidade:

- tech_overlap_count: métrica quantitativa de compatibilidade técnica (0‑20)
- sap_pair: flag binária para match SAP especializado (0/1)
- ingles_ok/espanhol_ok: flags de atendimento a requisitos de idioma (0/1)
- senioridade_gap: diferença ordinal entre senioridade candidato‑vaga (‑3 a +3)
- situacao_ord: posição ordinal no funil de recrutamento (0=Cadastrado, 5=Contratado)
- days_update: velocidade temporal de progressão no funil (dias)
- len_cv_pt_z: tamanho normalizado do CV como proxy de experiência

In [ ]:
# BLOCO A — CONFIGURAÇÃO INICIAL E CARREGAMENTO DOS DADOS
import json, re
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

BASE = Path.cwd()
DATA_DIR = BASE / "../data/" 

PATH_VAGAS = DATA_DIR / "vagas.json"
PATH_PROSPECTS = DATA_DIR / "prospects.json"
PATH_APPLICANTS = DATA_DIR / "applicants.json"

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

raw_vagas = load_json(PATH_VAGAS)
raw_prospects = load_json(PATH_PROSPECTS)
raw_applicants = load_json(PATH_APPLICANTS)

# BLOCO B — TRANSFORMAÇÃO DE DADOS HIERÁRQUICOS EM ESTRUTURA TABULAR
# Vagas (Jobs)
df_vagas = pd.json_normalize(list(raw_vagas.values()), sep='.')
df_vagas["vaga_id"] = list(map(str, raw_vagas.keys()))
cols_vagas = [
    "vaga_id",
    "informacoes_basicas.titulo_vaga",
    "informacoes_basicas.vaga_sap",
    "informacoes_basicas.cliente",
    "perfil_vaga.nivel_profissional",
    "perfil_vaga.nivel_ingles",
    "perfil_vaga.nivel_espanhol",
    "perfil_vaga.cidade",
    "perfil_vaga.estado",
    "perfil_vaga.pais",
    "perfil_vaga.principais_atividades",
    "perfil_vaga.competencia_tecnicas_e_comportamentais",
]
df_vagas = df_vagas[[c for c in cols_vagas if c in df_vagas.columns]]

# Prospects (por vaga)
rows = []
for vaga_id, payload in raw_prospects.items():
    for p in payload.get("prospects", []):
        r = p.copy()
        r["vaga_id"] = str(vaga_id)
        rows.append(r)
df_prospects = pd.DataFrame(rows).rename(columns={"codigo":"codigo_candidato","situacao_candidado":"situacao_candidato"})

# Applicants (candidatos)
df_app = pd.json_normalize(list(raw_applicants.values()), sep='.')
df_app["codigo_candidato"] = list(map(str, raw_applicants.keys()))
if "infos_basicas.codigo_profissional" in df_app.columns:
    df_app["codigo_candidato"] = df_app["infos_basicas.codigo_profissional"].fillna(df_app["codigo_candidato"]).astype(str)

cols_app = [
    "codigo_candidato",
    "infos_basicas.nome",
    "informacoes_profissionais.titulo_profissional",
    "informacoes_profissionais.area_atuacao",
    "formacao_e_idiomas.nivel_academico",
    "formacao_e_idiomas.nivel_ingles",
    "formacao_e_idiomas.nivel_espanhol",
    "informacoes_profissionais.conhecimentos_tecnicos",
    "cv_pt",
]
df_app = df_app[[c for c in cols_app if c in df_app.columns]]

# Join final
df = df_prospects.merge(df_vagas, on="vaga_id", how="left").merge(df_app, on="codigo_candidato", how="left")

# BLOCO C — PRÉ-PROCESSAMENTO CORRIGIDO
df = df.drop_duplicates(subset=["vaga_id","codigo_candidato"], keep="first")
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()

# Normalização de valores conhecidos
if "informacoes_basicas.vaga_sap" in df.columns:
    df["informacoes_basicas.vaga_sap"] = df["informacoes_basicas.vaga_sap"].str.lower().map(
        {"sim":"Sim","yes":"Sim","true":"Sim","não":"Não","nao":"Não","no":"Não"}
    ).fillna("Não")

# Numérica simples
df["len_cv_pt"] = df.get("cv_pt","").astype(str).str.len()
df["len_cv_pt"] = pd.to_numeric(df["len_cv_pt"], errors="coerce").fillna(0)

# One-hot apenas em baixa cardinalidade
low_card = [
    "situacao_candidato",
    "informacoes_basicas.vaga_sap",
    "perfil_vaga.nivel_profissional",
    "perfil_vaga.nivel_ingles",
    "perfil_vaga.nivel_espanhol",
]
for c in low_card:
    if c in df.columns:
        df[c] = df[c].replace({"": np.nan}).fillna("Desconhecido")

df_enc = pd.get_dummies(df, columns=[c for c in low_card if c in df.columns], drop_first=True, dtype=np.int8)

# Normalização de numéricas
scaler = StandardScaler()
df_enc["len_cv_pt_z"] = scaler.fit_transform(df_enc[["len_cv_pt"]]).astype(np.float32)
df_enc = df_enc.drop(columns=["len_cv_pt"], errors="ignore")

# BLOCO D — FEATURES ENGINEERING COM MAPEAMENTO CORRIGIDO
def get_series(df_, col, default=""):
    return df_[col] if col in df_.columns else pd.Series([default] * len(df_), index=df_.index)

def norm_txt(s):
    s = "" if pd.isna(s) else str(s).lower()
    s = re.sub(r"[^a-z0-9áâãàéêíóôõúç\+\#\.\- ]"," ",s)
    return re.sub(r"\s+"," ",s).strip()

TECH_TERMS = ["sap","abap","hana","sql","python","java",".net","c#","node","aws","azure","gcp","linux","docker","kubernetes","oracle","mysql","postgres","power bi","tableau"]
def extract_terms(text):
    t = norm_txt(text)
    return {tkn for tkn in TECH_TERMS if tkn in t}

# Overlap técnico e SAP
vaga_txt = (get_series(df, "perfil_vaga.principais_atividades").astype(str) + " " +
            get_series(df, "perfil_vaga.competencia_tecnicas_e_comportamentais").astype(str))
cand_txt = get_series(df, "informacoes_profissionais.conhecimentos_tecnicos").astype(str) + " " + \
           get_series(df, "cv_pt").astype(str)

vt = vaga_txt.map(extract_terms)
ct = cand_txt.map(extract_terms)
df_enc["tech_overlap_count"] = [len(a.intersection(b)) for a,b in zip(vt,ct)]

df_enc["is_sap_vaga"] = get_series(df, "informacoes_basicas.vaga_sap","Não").eq("Sim").astype(np.int8)
df_enc["cand_has_sap"] = cand_txt.str.contains(r"\bsap\b", regex=True, na=False).astype(np.int8)
df_enc["sap_pair"] = (df_enc["is_sap_vaga"] & df_enc["cand_has_sap"]).astype(np.int8)

# Idiomas
LANG_MAP = {"basico":1,"básico":1,"a1":1,"a2":2,"intermediario":3,"intermediário":3,"b1":3,"b2":4,"avancado":5,"avançado":5,"fluente":6,"c1":6,"c2":7}
def lang_rank(s):
    s = norm_txt(s)
    m = re.search(r"\b([abc][12])\b", s)
    if m:
        return LANG_MAP.get(m.group(1),0)
    for k,v in LANG_MAP.items():
        if k in s:
            return v
    return 0

df_enc["vaga_ing_rank"] = get_series(df, "perfil_vaga.nivel_ingles").astype(str).apply(lang_rank).astype("Int8")
df_enc["vaga_esp_rank"] = get_series(df, "perfil_vaga.nivel_espanhol").astype(str).apply(lang_rank).astype("Int8")
df_enc["cand_ing_rank"] = get_series(df, "formacao_e_idiomas.nivel_ingles").astype(str).apply(lang_rank).astype("Int8")
df_enc["cand_esp_rank"] = get_series(df, "formacao_e_idiomas.nivel_espanhol").astype(str).apply(lang_rank).astype("Int8")
df_enc["ingles_ok"] = (df_enc["cand_ing_rank"] >= df_enc["vaga_ing_rank"]).astype(np.int8)
df_enc["espanhol_ok"] = (df_enc["cand_esp_rank"] >= df_enc["vaga_esp_rank"]).astype(np.int8)

# Senioridade
def sen_vaga(s):
    s = norm_txt(s)
    return 3 if "sen" in s else 2 if "plen" in s else 1 if "jun" in s else 0

def sen_cand_from_title(s):
    s = norm_txt(s)
    return 3 if "sen" in s else 2 if "plen" in s else 1 if "jun" in s else 0

df_enc["vaga_sen_rank"] = get_series(df, "perfil_vaga.nivel_profissional").astype(str).apply(sen_vaga).astype("Int8")
df_enc["cand_sen_rank"] = get_series(df, "informacoes_profissionais.titulo_profissional").astype(str).apply(sen_cand_from_title).astype("Int8")
df_enc["senioridade_gap"] = (df_enc["cand_sen_rank"] - df_enc["vaga_sen_rank"]).astype("Int8")
df_enc["senioridade_ok"] = (df_enc["cand_sen_rank"] >= df_enc["vaga_sen_rank"]).astype(np.int8)

# Funil e tempo com MAPEAMENTO CORRIGIDO
def to_date(s): 
    return pd.to_datetime(s, dayfirst=True, errors="coerce")

df_enc["dt_cand"] = get_series(df, "data_candidatura").apply(to_date)
df_enc["dt_ult"] = get_series(df, "ultima_atualizacao").apply(to_date)
df_enc["days_update"] = (df_enc["dt_ult"] - df_enc["dt_cand"]).dt.days.fillna(0).astype(np.int16)

# CORREÇÃO CRÍTICA: Mapeamento robusto de situações de contratação
def map_situacao_ordinal(situacao_str):
    """Mapeia situações do candidato para valores ordinais, incluindo variações de 'Contratado'"""
    if pd.isna(situacao_str):
        return 2  # Default para "Encaminhado ao Requisitante"
    
    situacao_clean = str(situacao_str).lower().strip()
    
    # Mapeamento robusto que captura todas as variações
    if "contratado" in situacao_clean:  # Captura "Contratado" E "Contratado pela Decision"
        return 5
    elif "aprovado" in situacao_clean:
        return 4
    elif "entrevista" in situacao_clean:
        return 3
    elif "encaminhado" in situacao_clean:
        return 2
    elif "contato" in situacao_clean:
        return 1
    elif "cadastrado" in situacao_clean:
        return 0
    elif "reprovado" in situacao_clean:
        return 6
    else:
        return 2  # Default para situações não mapeadas

df_enc["situacao_ord"] = get_series(df, "situacao_candidato").apply(map_situacao_ordinal).astype("Int8")

# Binning e interação principal
len_cv_pt_raw = get_series(df, "cv_pt").astype(str).str.len()
df_enc["len_cv_bin"] = pd.qcut(len_cv_pt_raw.rank(method="first"), q=4, labels=False, duplicates="drop").astype("Int8")
df_enc["ok_eng_sen"] = (df_enc["ingles_ok"] & df_enc["senioridade_ok"]).astype(np.int8)

# BLOCO E — SELEÇÃO FINAL E PERSISTÊNCIA
keys = [c for c in ["vaga_id","codigo_candidato"] if c in df.columns]
features = [
    "tech_overlap_count","cand_has_sap","is_sap_vaga","sap_pair",
    "ingles_ok","espanhol_ok","vaga_ing_rank","cand_ing_rank","vaga_esp_rank","cand_esp_rank",
    "vaga_sen_rank","cand_sen_rank","senioridade_gap","senioridade_ok",
    "days_update","situacao_ord","len_cv_bin","ok_eng_sen","len_cv_pt_z"
]

df_final = pd.concat([df[keys], df_enc[features]], axis=1)

# Downcast para reduzir tamanho
for c in df_final.select_dtypes(include=["int64","int32","float64"]):
    df_final[c] = pd.to_numeric(df_final[c], downcast="integer")

OUT = BASE / "../data/df_clean.csv"
df_final.to_csv(OUT, index=False, encoding="utf-8")

# Verificação de candidatos contratados
print("Verificação da correção:")
print(f"Total de registros: {len(df_final)}")
print(f"Distribuição situacao_ord:\n{df_final['situacao_ord'].value_counts().sort_index()}")
print(f"Candidatos contratados (situacao_ord=5): {(df_final['situacao_ord']==5).sum()}")

OUT, df_final.shape


Verificação da correção:
Total de registros: 53759
Distribuição situacao_ord:
situacao_ord
2    43496
3     1048
4     6231
5     2984
Name: count, dtype: Int64
Candidatos contratados (situacao_ord=5): 2984


(WindowsPath('c:/Users/Dell/datathon/streamlit/notebooks/../data/df.csv'),
 (53759, 21))